In [ ]:
def int_plot():
    for n, pair in enumerate(exc_tiffs):
    x = np.linspace(-50, 50, 500)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    ax.plot(x, stats.norm.pdf(x, 0, std[0][n]), color=colours[0], label=f'Confocal: {np.round(std[0][n]*0.054, 2)} um')
    ax.plot(x, stats.norm.pdf(x, 0, std[1][n]), color=colours[1], label=f'RLTV: {np.round(std[1][n]*0.054, 2)} um')
    ax.plot(x, stats.norm.pdf(x, 0, std[2][n]), color=colours[2], label=f'AI: {np.round(std[2][n]*0.054, 2)} um')
    ax.legend()
    fig.savefig(f'{pair[0]}.png')
    print(np.round(xz['fit'].tolist()[1]*1.5,2))
    print(np.round(yz['fit'].tolist()[1]*1.5,2))
    print(np.round(xy['fit'].tolist()[1]*0.054,2))

In [ ]:
#2d plots
import pandas as pd
from libtiff import TIFF
from imutils import contours
from skimage import measure
import imutils
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage import draw
from scipy import signal
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from scipy.stats import multivariate_normal
from scipy import stats

exc_tiffs = []
#tiffs = ['sted_acut.tif','RLTV_cut.tif','conf_cut.tif','ai_cut.tif','axz.tif', 'ayz.tif', 'rlxz.tif','rlyz.tif','sxz.tif','syz.tif','cxz.tif','cyz.tif']
tiffs = ['C:\\Users\\antim\\Desktop\\ip3rcl\\rawxy.tif','C:\\Users\\antim\\Desktop\\ip3rcl\\rawxz.tif','C:\\Users\\antim\\Desktop\\ip3rcl\\rawyz.tif','C:\\Users\\antim\\Desktop\\ip3rcl\\rltvxy.tif','C:\\Users\\antim\\Desktop\\ip3rcl\\rltvxz.tif','C:\\Users\\antim\\Desktop\\ip3rcl\\rltvyz.tif','C:\\Users\\antim\\Desktop\\ip3rcl\\aixy.tif','C:\\Users\\antim\\Desktop\\ip3rcl\\aixz.tif','C:\\Users\\antim\\Desktop\\ip3rcl\\aiyz.tif']
#tiffs = ['C:\\Users\\antim\\Desktop\\spheres_supplement\\MAX_sted_no_11.tif']
for isobr in tiffs:
    tif = TIFF.open(isobr, mode='r')
    df1 = pd.DataFrame()

    for n, img in enumerate(tif.iter_images()): 

        blurred = cv2.GaussianBlur(img, (5, 5), 0)

        thresh = cv2.threshold(blurred, blurred.max()*0.7, blurred.max(), cv2.THRESH_BINARY)[1]
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=3)
        thresh = np.where(thresh>0, 1, 0)
        labels = measure.label(thresh, connectivity=2)
        mask = np.zeros(thresh.shape, dtype="uint8")
        for label in np.unique(labels):
            if label == 0:
                continue
            labelMask = np.zeros(thresh.shape, dtype="uint8")
            labelMask[labels == label] = 255
            numPixels = cv2.countNonZero(labelMask)
            if numPixels > 80:
                mask = cv2.add(mask, labelMask)
            
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        if bool(cnts):
            cnts = contours.sort_contours(cnts)[0]
            (x, y, w, h) = cv2.boundingRect(cnts[0])
            lineim = cv2.line(img.copy(), (x, y+h-1), (x+w-1, y), (255, 255, 255), 2)
            #lineim = cv2.line(img, (x, y+h-1), (x+w-1, y), (255, 255, 255), 2)
            line = np.transpose(np.array(draw.line(x, y+h-1, x+w-1, y)))
            data = img[line[:, 1], line[:, 0]]
            df1[f'cont{n}'] = pd.Series(signal.resample((data),35))
            #plt.figure('Intensity profile')
            #plt.plot(signal.resample((data),35))
            #plt.draw()
            #cv2.drawContours(img, cnts, -1, (51155,51155,51155), 3)
            #imS = cv2.resize(img, (1920, 1080)) 
            #cv2.imshow('Image', imS)
            #cv2.imshow('Image', img)
            #cv2.waitKey(0)


    mean,std=norm.fit(df1.mean(axis=1))
    df1['mean'] = df1.mean(axis=1)
    df1['fit'] = pd.Series([mean, std])
    df1.to_excel(f'{isobr}.xlsx')
    


#exc_tiffs = [['C:\\Users\\antim\\Desktop\\spheres_supplement\\xz_sted.tif.xlsx', 'C:\\Users\\antim\\Desktop\\spheres_supplement\\yz_sted.tif.xlsx'],
      #   ['C:\\Users\\antim\\Desktop\\spheres_supplement\\MAX_xz_conf.tif.xlsx', 'C:\\Users\\antim\\Desktop\\spheres_supplement\\MAX_yz_conf.tif.xlsx']]
exc_tiffs = [['C:\\Users\\antim\\Desktop\\ip3rcl\\rawxz.tif.xlsx','C:\\Users\\antim\\Desktop\\ip3rcl\\rawyz.tif.xlsx', 'C:\\Users\\antim\\Desktop\\ip3rcl\\rawxy.tif.xlsx'],['C:\\Users\\antim\\Desktop\\ip3rcl\\rltvxz.tif.xlsx','C:\\Users\\antim\\Desktop\\ip3rcl\\rltvyz.tif.xlsx', 'C:\\Users\\antim\\Desktop\\ip3rcl\\rltvxy.tif.xlsx'],['C:\\Users\\antim\\Desktop\\ip3rcl\\aixz.tif.xlsx','C:\\Users\\antim\\Desktop\\ip3rcl\\aiyz.tif.xlsx','C:\\Users\\antim\\Desktop\\ip3rcl\\aixy.tif.xlsx']]
colours = [(1, 0, 0), (0,1,0), (1,0,1)]

def twodplit(sigma, colour):
    x = np.linspace(-25, 25, 100)
    plt.plot(x, stats.norm.pdf(x, 0, sigma), color=colour, label=f'FWHM: {sigma} um')
std=[]
for n, pair in enumerate(exc_tiffs):
    xz = pd.read_excel(pair[0])
    yz = pd.read_excel(pair[1])
    xy = pd.read_excel(pair[2])
    xz.dropna(subset = ['fit'], inplace=True)
    yz.dropna(subset = ['fit'], inplace=True)
    xy.dropna(subset = ['fit'], inplace=True)
    muxz, stdxz = 0, xz['fit'].tolist()[1]
    muyz, stdyz = 0, yz['fit'].tolist()[1]
    muxy, stdxy = 0, xy['fit'].tolist()[1]
    std.append([stdxy,stdxz,stdyz])
for n, pair in enumerate(exc_tiffs):
    x = np.linspace(-50, 50, 500)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    ax.plot(x, stats.norm.pdf(x, 0, std[0][n]), color=colours[0], label=f'Confocal: {np.round(std[0][n]*0.054, 2)} um')
    ax.plot(x, stats.norm.pdf(x, 0, std[1][n]), color=colours[1], label=f'RLTV: {np.round(std[1][n]*0.054, 2)} um')
    ax.plot(x, stats.norm.pdf(x, 0, std[2][n]), color=colours[2], label=f'AI: {np.round(std[2][n]*0.054, 2)} um')
    ax.legend()
    fig.savefig(f'{pair[0]}.png')
    print(np.round(xz['fit'].tolist()[1]*1.5,2))
    print(np.round(yz['fit'].tolist()[1]*1.5,2))
    print(np.round(xy['fit'].tolist()[1]*0.054,2))

def threedplot(muxz, muyz, stdxz, stdyz):
    x, y = np.mgrid[-150.0:150.0:30j, -150.0:150.0:30j]
    xy = np.column_stack([x.flat, y.flat])
    mu = np.array([muxz,  muyz])
    sigma = np.array([stdxz, stdyz])

    covariance = np.diag(sigma**2)

    z = multivariate_normal.pdf(xy, mean=mu, cov=covariance)*99000

    z = z.reshape(x.shape)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot_surface(x,y,z, shade=False, facecolor=(0,0,0,0.0001), edgecolor=colours[n])
    ax.set_facecolor((0,0,0,0.0001))
    ax.set_xlabel('YZ FWHM')
    ax.set_ylabel('XZ FWHM')

def twodplit(sigma, colour):
    x = np.linspace(-25, 25, 100)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x, stats.norm.pdf(x, 0, sigma), color=colour, label=f'FWHM: {sigma} um')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats


mu = 0
# xy sigmaconf, sigmasted, sigmaai  = 2.14, 3.71, 2.62
# xz sigmaconf, sigmasted, sigmaai  = 18.77, 1.41, 4.28
sigmaconf, sigmasted, sigmaai  = 21.82, 1.56, 5.01
x = np.linspace(mu - 25, mu + 25, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigmasted), color='blue', label=f'STED: {sigmasted} um')
plt.plot(x, stats.norm.pdf(x, mu, sigmaconf), color='red', label=f'Confocal: {sigmaconf} um')
plt.plot(x, stats.norm.pdf(x, mu, sigmaai), color='#ff00ff', label=f'AI: {sigmaai} um')
plt.legend()
plt.show()


In [ ]:
#cubes
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
alpha = 0.8
colours = [[1,0,0, alpha],[0,0,1, alpha],[0,1,0, alpha], [1, 0, 1, alpha]]
#0.3*10 = 3
#xz, yz, xy
sizes = [[20, 20, 5], [6, 6, 4],[18, 14, 5],[12, 13, 3]]

for n, i in enumerate(sizes):
    data = np.ones(i, dtype=bool)
    colors = np.empty(i + [4], dtype=np.float32)
    colors[:] = colours[n]
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim(0,25)
    ax.set_ylim(0,25)
    ax.set_zlim(0,2.5)
    ax.invert_yaxis()
    zticks = np.arange(0, 7, 1)
    zlabels = [f'{z*100}' for z in zticks]
    ax.set_zticks(zticks, labels=zlabels)
    yticks = np.arange(0, 30, 5)
    ylabels = [f'{y/10}' for y in yticks]
    ax.set_yticks(yticks, labels=ylabels)
    xticks = np.arange(0, 30, 5)
    xlabels = [f'{x/10}' for x in xticks]
    ax.set_xticks(xticks, labels=xlabels)
    ax.set_xlabel('XZ Resolution, $\mu$m')
    ax.set_ylabel('YZ Resolution, $\mu$m')
    ax.set_zlabel('XY Resolution, nm')
    ax.voxels(data, facecolors=colors)
    fig.savefig(f'{i}.png')
    #plt.tight_layout(pad=0.4, w_pad=1.5, h_pad=1.0)

In [ ]:
#raw fwhm
import pandas as pd
from libtiff import TIFF
from imutils import contours
from skimage import measure
import imutils
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage import draw
from scipy import signal

tif = TIFF.open('cluster_try.tif', mode='r')
df1 = pd.DataFrame()

for img in tif.iter_images(): 

	blurred = cv2.GaussianBlur(img, (5, 5), 0)

	thresh = cv2.threshold(blurred, blurred.max()*0.6, blurred.max(), cv2.THRESH_BINARY)[1]
	thresh = cv2.erode(thresh, None, iterations=2)
	thresh = cv2.dilate(thresh, None, iterations=3)
	thresh = np.where(thresh>0, 1, 0)
	labels = measure.label(thresh, connectivity=2)
	mask = np.zeros(thresh.shape, dtype="uint8")
	for label in np.unique(labels):
		if label == 0:
			continue
		labelMask = np.zeros(thresh.shape, dtype="uint8")
		labelMask[labels == label] = 255
		numPixels = cv2.countNonZero(labelMask)
		if numPixels > 30:
			mask = cv2.add(mask, labelMask)
		
	cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	cnts = imutils.grab_contours(cnts)
	if bool(cnts):
		cnts = contours.sort_contours(cnts)[0]
		for n, contour in enumerate(cnts):
			
			(x, y, w, h) = cv2.boundingRect(contour)
			
			lineim = cv2.line(img.copy(), (x, y), (x+w, y+h), (51255, 51255, 51255), 2)
			line = np.transpose(np.array(draw.line(x, y, x+w, y+h)))
			
			data = img[line[:, 1], line[:, 0]]
			df1[f'cont{n}'] = pd.Series(signal.resample((data),35))
			plt.figure('Intensity profile')
			plt.plot(signal.resample((data),35))
			

			plt.draw()

			#plt.imshow(img[y:y+h, x:x+w])
		cv2.drawContours(img, cnts, -1, (51155,51155,51155), 3)
		imS = cv2.resize(img, (1920, 1080)) 
		cv2.imshow('Image', imS)
		cv2.waitKey(0)




df1.to_excel('output.xlsx')

In [ ]:
#peak SNR and regularization constant

from libtiff import TIFF
from imutils import contours
from skimage import measure
import imutils
import numpy as np
import cv2
import matplotlib.pyplot as plt


tif = TIFF.open('MAX_30_vbnfa_no.tif', mode='r')


for img in tif.iter_images(): 
	#img = cv2.imread('21_vbnfa_ai.tif', cv2.COLOR_BGR2GRAY)
	#img = cv2.imread('PSF RW.tif','21_vbnfa_ai.tif' cv2.IMREAD_UNCHANGED)
	blurred = cv2.GaussianBlur(img, (3, 3), 0)
	#print(f'dtype: {img.dtype}, shape: {img.shape}, min: {np.min(img)}, max: {np.max(img)}')
	thresh = cv2.threshold(blurred, blurred.max()*0.6, blurred.max(), cv2.THRESH_BINARY)[1]
	thresh = cv2.erode(thresh, None, iterations=2)
	thresh = cv2.dilate(thresh, None, iterations=4)
	thresh = np.where(thresh>0, 1, 0)
	labels = measure.label(thresh, connectivity=2)
	mask = np.zeros(thresh.shape, dtype="uint8")
	for label in np.unique(labels):
		if label == 0:
			continue
		labelMask = np.zeros(thresh.shape, dtype="uint8")
		labelMask[labels == label] = 255
		numPixels = cv2.countNonZero(labelMask)
		if numPixels > 30:
			mask = cv2.add(mask, labelMask)
		
	cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	cnts = imutils.grab_contours(cnts)
	if bool(cnts):
		cnts = contours.sort_contours(cnts)[0]
		(x, y, w, h) = cv2.boundingRect(cnts[0])
		SNR = np.array([])
		regular = np.array([])
		for i in range(x+1, x+h-1):
			for j in range(y+1, y+h-1):
				if np.std(img[j:j+2, i:i+2]) !=0:
					SNR = np.append(SNR, np.average(img[j:j+2, i:i+2])/np.std(img[j:j+2, i:i+2]))
					
		regular = np.append(regular, 50/np.max(SNR))
		#plt.imshow(img[y:y+h, x:x+w])
		cv2.drawContours(img, cnts, 0, (250,250,250), 3)
		cv2.imshow('Image', img)
		cv2.waitKey(0)
print(SNR)
print(max(SNR))
print(np.min(regular))
#cv2.imshow('Image', img)
#cv2.drawContours(img, cnts, 0, (25,25,25), 3)
#plt.imshow(img)
#cv2.waitKey(0)